In [1]:
import csv
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
import warnings
import gensim
from gensim.models import Word2Vec
import pandas as pd

from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
import gensim
import sys
import numpy as np
import pandas as pd
import string
import logging
import random
import keras
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

### in_domain_train dataframe

In [8]:
# in_domain_train = open("../dataset/cola_public_1.1/cola_public/raw/in_domain_train.tsv")
# in_domain_dev = open("../dataset/cola_public_1.1/cola_public/raw/in_domain_dev.tsv")
# in_domain_test = open("../dataset/cola-in-domain-open-evaluation/cola_in_domain_test.tsv")

In [2]:
in_domain_train_tokenized = open("../dataset/cola_public_1.1/cola_public/tokenized/in_domain_train.tsv")
read_train = csv.reader(in_domain_train_tokenized, delimiter="\t")
train = []
for row in read_train:
#     print(row)
    train.append(row)
in_domain_train_tokenized.close()

In [3]:
train_array = np.array(train)
train_df = pd.DataFrame(train_array)

In [4]:
train_df

,0,1,2,3
0,gj04,1,,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,,one more pseudo generalization and i 'm giving...
2,gj04,1,,one more pseudo generalization or i 'm giving ...
3,gj04,1,,"the more we study verbs , the crazier they get ."
4,gj04,1,,day by day the facts are getting murkier .
...,...,...,...,...
8546,ad03,0,*,poseidon appears to own a dragon
8547,ad03,0,*,digitize is my happiest memory
8548,ad03,1,,it is easy to slay the gorgon .
8549,ad03,1,,i had the strangest feeling that i knew you .


### W2V model od liste reci iz recenica

In [52]:
# list_sent = [row[3] for row in train]

In [5]:
list_sent_tkn = []

# for sent in list_sent:
for sent in train_df[3]:
    print(sent)
    print(word_tokenize(sent))
    list_sent_tkn.append(word_tokenize(sent))

our friends wo n't buy this analysis , let alone the next one we propose .
['our', 'friends', 'wo', "n't", 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
one more pseudo generalization and i 'm giving up .
['one', 'more', 'pseudo', 'generalization', 'and', 'i', "'m", 'giving', 'up', '.']
one more pseudo generalization or i 'm giving up .
['one', 'more', 'pseudo', 'generalization', 'or', 'i', "'m", 'giving', 'up', '.']
the more we study verbs , the crazier they get .
['the', 'more', 'we', 'study', 'verbs', ',', 'the', 'crazier', 'they', 'get', '.']
day by day the facts are getting murkier .
['day', 'by', 'day', 'the', 'facts', 'are', 'getting', 'murkier', '.']
i 'll fix you a drink .
['i', "'ll", 'fix', 'you', 'a', 'drink', '.']
fred watered the plants flat .
['fred', 'watered', 'the', 'plants', 'flat', '.']
bill coughed his way out of the restaurant .
['bill', 'coughed', 'his', 'way', 'out', 'of', 'the', 'restaurant', '.']
we 're dancing the

['there', 'are', "n't", 'many', 'linguistics', 'students', 'here', '.']
i have n't met many linguistics students .
['i', 'have', "n't", 'met', 'many', 'linguistics', 'students', '.']
what does every student buy ?
['what', 'does', 'every', 'student', 'buy', '?']
i need sally to be there .
['i', 'need', 'sally', 'to', 'be', 'there', '.']
the boat sank to collect the insurance .
['the', 'boat', 'sank', 'to', 'collect', 'the', 'insurance', '.']
the boat was sunk to collect the insurance .
['the', 'boat', 'was', 'sunk', 'to', 'collect', 'the', 'insurance', '.']
john wants to win .
['john', 'wants', 'to', 'win', '.']
the bed was unmade .
['the', 'bed', 'was', 'unmade', '.']
headway was unmade .
['headway', 'was', 'unmade', '.']
john was unknown .
['john', 'was', 'unknown', '.']
john was unknown to be the murderer .
['john', 'was', 'unknown', 'to', 'be', 'the', 'murderer', '.']
we knew john to be the murderer .
['we', 'knew', 'john', 'to', 'be', 'the', 'murderer', '.']
he fed the children .
[

near everyone angleton did , dulles stood .
['near', 'everyone', 'angleton', 'did', ',', 'dulles', 'stood', '.']
sally will stand near mag , but he wo n't holly .
['sally', 'will', 'stand', 'near', 'mag', ',', 'but', 'he', 'wo', "n't", 'holly', '.']
while holly did n't discuss a report about every boy , she did every girl .
['while', 'holly', 'did', "n't", 'discuss', 'a', 'report', 'about', 'every', 'boy', ',', 'she', 'did', 'every', 'girl', '.']
sally will stand near every woman that you will .
['sally', 'will', 'stand', 'near', 'every', 'woman', 'that', 'you', 'will', '.']
i know which woman holly will discuss a report about , but i do n't know which woman you will .
['i', 'know', 'which', 'woman', 'holly', 'will', 'discuss', 'a', 'report', 'about', ',', 'but', 'i', 'do', "n't", 'know', 'which', 'woman', 'you', 'will', '.']
sam stood near yesterday every one of the women we 'd been discussing .
['sam', 'stood', 'near', 'yesterday', 'every', 'one', 'of', 'the', 'women', 'we', "'d", 'b

which boy 's guardian 's did we elect employer president ?
['which', 'boy', "'s", 'guardian', "'s", 'did', 'we', 'elect', 'employer', 'president', '?']
how have you picked up tnt carelessly ?
['how', 'have', 'you', 'picked', 'up', 'tnt', 'carelessly', '?']
how carelessly have you picked up tnt ?
['how', 'carelessly', 'have', 'you', 'picked', 'up', 'tnt', '?']
sheila married that tall a man .
['sheila', 'married', 'that', 'tall', 'a', 'man', '.']
how tall a man did sheila marry ?
['how', 'tall', 'a', 'man', 'did', 'sheila', 'marry', '?']
how tall did sheila marry a man ?
['how', 'tall', 'did', 'sheila', 'marry', 'a', 'man', '?']
how did sheila marry tall a man ?
['how', 'did', 'sheila', 'marry', 'tall', 'a', 'man', '?']
on which bed does tom sleep ?
['on', 'which', 'bed', 'does', 'tom', 'sleep', '?']
the bed on which tom slept was hard .
['the', 'bed', 'on', 'which', 'tom', 'slept', 'was', 'hard', '.']
which bed did tom sleep on ?
['which', 'bed', 'did', 'tom', 'sleep', 'on', '?']
the b

['sally', 'might', 'be', 'pregnant', ',', 'and', 'i', 'believe', 'the', 'claim', 'that', 'sheila', 'definitely', 'is', 'pregnant', '.']
sally might be pregnant , and i believe that sheila definitely is pregnant .
['sally', 'might', 'be', 'pregnant', ',', 'and', 'i', 'believe', 'that', 'sheila', 'definitely', 'is', 'pregnant', '.']
sally is tall , and may be , and sheila is short , and definitely is , blond .
['sally', 'is', 'tall', ',', 'and', 'may', 'be', ',', 'and', 'sheila', 'is', 'short', ',', 'and', 'definitely', 'is', ',', 'blond', '.']
hank plays the guitar and finds arrangements for all the old folk songs which are still sung in these hills , and ernie writes down all the old folk songs which are still sung in these hills .
['hank', 'plays', 'the', 'guitar', 'and', 'finds', 'arrangements', 'for', 'all', 'the', 'old', 'folk', 'songs', 'which', 'are', 'still', 'sung', 'in', 'these', 'hills', ',', 'and', 'ernie', 'writes', 'down', 'all', 'the', 'old', 'folk', 'songs', 'which', 'ar

...

['the', 'truck', 'spread', 'salt', '.']
the truck spread the salt .
['the', 'truck', 'spread', 'the', 'salt', '.']
the truck spread salts .
['the', 'truck', 'spread', 'salts', '.']
this truck spread less salt than that one .
['this', 'truck', 'spread', 'less', 'salt', 'than', 'that', 'one', '.']
this truck spread fewer salt than that one .
['this', 'truck', 'spread', 'fewer', 'salt', 'than', 'that', 'one', '.']
there are fewer trucks on the motorway this winter .
['there', 'are', 'fewer', 'trucks', 'on', 'the', 'motorway', 'this', 'winter', '.']
there are less trucks on the motorway this winter .
['there', 'are', 'less', 'trucks', 'on', 'the', 'motorway', 'this', 'winter', '.']
the white rabbit vanished his watch .
['the', 'white', 'rabbit', 'vanished', 'his', 'watch', '.']
dogs chase cats .
['dogs', 'chase', 'cats', '.']
dogs chase .
['dogs', 'chase', '.']
flora cooks .
['flora', 'cooks', '.']
flora cooks gourmet meals .
['flora', 'cooks', 'gourmet', 'meals', '.']
the cat shot into th

you british and you americans pamper yourselves .
['you', 'british', 'and', 'you', 'americans', 'pamper', 'yourselves', '.']
you british or you americans will get yourselves in trouble .
['you', 'british', 'or', 'you', 'americans', 'will', 'get', 'yourselves', 'in', 'trouble', '.']
you and kerry have outdone yourselves .
['you', 'and', 'kerry', 'have', 'outdone', 'yourselves', '.']
you or kerry have perjured yourselves .
['you', 'or', 'kerry', 'have', 'perjured', 'yourselves', '.']
the boys and the girls seem happy .
['the', 'boys', 'and', 'the', 'girls', 'seem', 'happy', '.']
the boys and the girls seems happy .
['the', 'boys', 'and', 'the', 'girls', 'seems', 'happy', '.']
either the boys or the girls are going to be there .
['either', 'the', 'boys', 'or', 'the', 'girls', 'are', 'going', 'to', 'be', 'there', '.']
the students and professor swansong are meeting in the park .
['the', 'students', 'and', 'professor', 'swansong', 'are', 'meeting', 'in', 'the', 'park', '.']
either professor

['mary', 'wo', "n't", 'have', 'been', 'eating', 'cake', ',', 'but', 'john', '.']
mary wo n't have been eating cake , but john will .
['mary', 'wo', "n't", 'have', 'been', 'eating', 'cake', ',', 'but', 'john', 'will', '.']
mary wo n't have been eating cake , but john will have .
['mary', 'wo', "n't", 'have', 'been', 'eating', 'cake', ',', 'but', 'john', 'will', 'have', '.']
mary wo n't have been eating cake , but john will have been .
['mary', 'wo', "n't", 'have', 'been', 'eating', 'cake', ',', 'but', 'john', 'will', 'have', 'been', '.']
john will enthusiastically have been eating cake .
['john', 'will', 'enthusiastically', 'have', 'been', 'eating', 'cake', '.']
john will have enthusiastically been eating cake .
['john', 'will', 'have', 'enthusiastically', 'been', 'eating', 'cake', '.']
john will have been eating enthusiastically cake .
['john', 'will', 'have', 'been', 'eating', 'enthusiastically', 'cake', '.']
john will have been eating cake enthusiastically .
['john', 'will', 'have', 

['bill', 'did', 'not', 'destroy', 'the', 'world', '.']
both the twins might have been at the party .
['both', 'the', 'twins', 'might', 'have', 'been', 'at', 'the', 'party', '.']
that plato loved aster was obvious .
['that', 'plato', 'loved', 'aster', 'was', 'obvious', '.']
the pigs grunt
['the', 'pigs', 'grunt']
where place are you living .
['where', 'place', 'are', 'you', 'living', '.']
who was for medea to poison awful ?
['who', 'was', 'for', 'medea', 'to', 'poison', 'awful', '?']
julie maintained if the barman was sober .
['julie', 'maintained', 'if', 'the', 'barman', 'was', 'sober', '.']
the analysis of lucy took longer than that of gomez .
['the', 'analysis', 'of', 'lucy', 'took', 'longer', 'than', 'that', 'of', 'gomez', '.']
julie became a fond .
['julie', 'became', 'a', 'fond', '.']
i climbed up the tree .
['i', 'climbed', 'up', 'the', 'tree', '.']
i inquired when we could leave .
['i', 'inquired', 'when', 'we', 'could', 'leave', '.']
where alison and david soaked their feet was

anson demonized david at the club .
['anson', 'demonized', 'david', 'at', 'the', 'club', '.']
jason asked whether the potion was ready
['jason', 'asked', 'whether', 'the', 'potion', 'was', 'ready']
frieda closed the door
['frieda', 'closed', 'the', 'door']
peter is the old pigs .
['peter', 'is', 'the', 'old', 'pigs', '.']
medea might have given jason a poisoned robe ( just treat a poisoned robe as an np
['medea', 'might', 'have', 'given', 'jason', 'a', 'poisoned', 'robe', '(', 'just', 'treat', 'a', 'poisoned', 'robe', 'as', 'an', 'np']
quickly kiss anson !
['quickly', 'kiss', 'anson', '!']
anson believed jenny to have hurt himself .
['anson', 'believed', 'jenny', 'to', 'have', 'hurt', 'himself', '.']
julie felt hot
['julie', 'felt', 'hot']
agamemnon expected esther to seem to be happy .
['agamemnon', 'expected', 'esther', 'to', 'seem', 'to', 'be', 'happy', '.']
him book
['him', 'book']
that the answer is obvious upset hermes .
['that', 'the', 'answer', 'is', 'obvious', 'upset', 'hermes

In [6]:
list_sent_tkn[:2]

[['our',
  'friends',
  'wo',
  "n't",
  'buy',
  'this',
  'analysis',
  ',',
  'let',
  'alone',
  'the',
  'next',
  'one',
  'we',
  'propose',
  '.'],
 ['one',
  'more',
  'pseudo',
  'generalization',
  'and',
  'i',
  "'m",
  'giving',
  'up',
  '.']]

In [7]:
len(list_sent_tkn)

8551

In [26]:
w2v_model = Word2Vec(list_sent_tkn, min_count=1, vector_size= 50, workers=3, window =3, sg = 1)

In [27]:
w2v_model.wv[',']

array([-3.2026219e-01,  1.1816813e-01, -1.3775475e-01, -4.0295497e-01,
        1.2255760e-01,  5.1795198e-03,  8.3973378e-02,  6.1494384e-02,
       -4.2515928e-01,  4.9880479e-02,  9.2024944e-05, -1.2041519e-01,
       -4.5509733e-02,  5.2276903e-01, -1.0924996e-01, -1.2869306e-01,
       -2.2270533e-01,  2.8285241e-01, -1.1590481e-01, -3.1940374e-01,
        1.5849450e-01, -3.7274748e-01,  7.0590901e-01,  2.9186415e-02,
       -3.4472731e-01, -4.7956708e-01,  2.7151749e-01,  2.7901715e-02,
       -3.7569907e-01,  2.4062517e-01,  4.8862416e-02, -6.5923460e-02,
        1.3947928e-01, -4.9761429e-01,  3.8827679e-01, -3.1730887e-01,
        5.7822358e-02,  4.5746320e-01,  1.3590657e-02, -3.5298496e-01,
        2.8222582e-01,  7.3518664e-02, -3.9024684e-01,  8.1420183e-01,
        4.9347934e-01, -3.8806283e-01,  1.8906182e-01,  4.2580221e-02,
        4.6742842e-01, -5.6758426e-02], dtype=float32)

In [28]:
len(w2v_model.wv)

5414

In [29]:
w2v_weights = w2v_model.wv.vectors
vocab_size, embedding_size = w2v_weights.shape

In [15]:
print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))

Vocabulary Size: 5414 - Embedding Dim: 50


In [31]:
def word2token(word):
    try:
        return w2v_model.wv.vocab[word].index
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
def token2word(token):
    return w2v_model.wv.index2word[token]

In [34]:
# Create an iterator that formats data from the dataset proper for
# LSTM training

# Sequences will be padded or truncated to this length
MAX_SEQUENCE_LENGTH = 200

# Samples of categories with less than this number of samples will be ignored
DROP_THRESHOLD = 10000

class SequenceIterator:
    def __init__(self, dataset, drop_threshold, seq_length):
        self.dataset = dataset

        self.translator = str.maketrans('', '', string.punctuation + '–')
        self.categories, self.ccount = np.unique(dataset[1], return_counts=True)
        
        self.seq_length = seq_length
        
        # Samples of categories with less than this number of samples will be ignored
        self.drop_categos = []
        for cat, count in zip(self.categories, self.ccount):
            if count < drop_threshold:
                self.drop_categos.append(cat)
        
        # Remaining categories
        self.categories = np.setdiff1d(self.categories, self.drop_categos)
        
    def __iter__(self):
        for news, cat in zip(self.dataset.iloc[:, 0], self.dataset.iloc[:, 1]):
            if cat in self.drop_categos:
                continue
            
            # Make all characters lower-case
            news = news.lower()
            
            # Clean string of all punctuation
            news = news.translate(self.translator)

            words = np.array([word2token(w) for w in news.split(' ')[:self.seq_length] if w != ''])
                                
            yield (words, cat)

In [38]:
df = train_df.iloc[:, [1, 3]]

In [39]:
df.head()

,1,3
0,1,"our friends wo n't buy this analysis , let alo..."
1,1,one more pseudo generalization and i 'm giving...
2,1,one more pseudo generalization or i 'm giving ...
3,1,"the more we study verbs , the crazier they get ."
4,1,day by day the facts are getting murkier .


In [40]:
sequences = SequenceIterator(train_df, DROP_THRESHOLD, MAX_SEQUENCE_LENGTH)

# Used for generating the labels in the set
cat_dict = {k: v for k, v in zip(sequences.categories, range(len(sequences.categories)))}

set_x = []
set_y = []
for w, c in sequences:
    set_x.append(w)
    set_y.append(cat_dict[c])
    
# Padding sequences with 0.
set_x = pad_sequences(set_x, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)
set_y = np.array(set_y)

print(set_x.shape)
print(set_y.shape)

(0, 200)
(0,)


In [24]:
X = np.array(train_df[3].tolist())
y = np.array(train_df[1].tolist())

leinst = LabelEncoder()
yinst = leinst.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(X, yinst, test_size=0.2, random_state=42)
# x_train = x_train.reshape(len(x_train),117, 117, 3)
# x_test = x_test.reshape(len(x_test),117, 117, 3)
print(x_train.shape)
print(x_test.shape)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, random_state=42)
# x_val = x_val.reshape(len(x_val),117, 117, 3)
print(x_val.shape)
print(x_test.shape)


n_categories = len(leinst.classes_)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

(6840,)
(1711,)
(855,)
(856,)


In [41]:
x_train

array(['john seemed a fool .', 'it is of physics .',
       'that automobile factories abound in michigan worries me greatly .',
       ..., 'he looks more formidable than he is .',
       "sally might have eaten cabbage , but holly should n't .",
       'i think that you will see that the girl will put a picture on your desk .'],
      dtype='<U233')

In [25]:
model = Sequential()

# Keras Embedding layer with Word2Vec weights initialization
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    weights=[w2v_weights],
                    input_length=MAX_SEQUENCE_LENGTH,
                    mask_zero=True,
                    trainable=False))

model.add(Bidirectional(LSTM(100)))
model.add(Dense(n_categories, activation='softmax'))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 50)           270700    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 2)                 402       
                                                                 
Total params: 391,902
Trainable params: 121,202
Non-trainable params: 270,700
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=5, batch_size=64,
                    validation_data=(val_x, val_y), verbose=1)

In [20]:
model1 = Word2Vec(sentences=list_sent_tkn, vector_size=300, window=10, min_count=1)
words = list(model1.wv.index_to_key)
# print(words)
# print(len(words))

In [21]:
len(words)

5414

In [57]:
word_vec_dict={}
for word in words:
    word_vec_dict[word]=model1.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size

The no of key-value pairs :  5414


In [58]:
w2v_weights = model1.wv.vectors
vocab_size, embedding_size = w2v_weights.shape
print(vocab_size, embedding_size)
MAX_SEQUENCE_LENGTH = 200

5414 300


In [59]:



maxi=-1
for i,rev in enumerate(train_df[3]):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)


tok = Tokenizer(filters='')
tok.fit_on_texts(train_df[3])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(train_df[3])

print(vocab_size)

5415


In [60]:
# now padding to have a amximum length of 44
pad_rev= pad_sequences(encd_rev, maxlen=maxi+1, padding='post')
pad_rev.shape   # note that we had 100K reviews and we have padded each review to have  a lenght of 1565 words.
print(pad_rev)
print(pad_rev.shape)

[[ 228  275  259 ...    0    0    0]
 [  83   30 2614 ...    0    0    0]
 [  83   30 2614 ...    0    0    0]
 ...
 [  17    7  186 ...    0    0    0]
 [   5   44    2 ...    0    0    0]
 [  49   59   21 ...    0    0    0]]
(8551, 45)


In [61]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embedding_size))
# print(len(tok.word_index.items()))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

train_size=0.8
# print(embed_matrix.shape)
# X = train_df[3]
X = np.array(train_df[3].tolist())
y = np.array(train_df[1].tolist())

leinst = LabelEncoder()
yinst = leinst.fit_transform(y)

ycat = to_categorical(train_df[1])  # one hot target as required by NN
# print(ycat)
# X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_train, X_test, y_train, y_test = train_test_split(pad_rev, ycat, test_size=0.2, shuffle=True, random_state=42)

# test_size = 0.5
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=42)


print(X_train.shape), print(y_train.shape)
print(X_validation.shape), print(y_validation.shape)
print(X_test.shape), print(y_test.shape)

(6840, 45)
(6840, 2)
(856, 45)
(856, 2)
(855, 45)
(855, 2)


(None, None)

In [62]:

num_classes = len(leinst.classes_)
print(num_classes)
print(leinst.classes_)

2
['0' '1']


In [63]:

print(X_train)

[[   8  372    4 ...    0    0    0]
 [  17    7   10 ...    0    0    0]
 [   6 4818 4819 ...    0    0    0]
 ...
 [  11 1576   30 ...    0    0    0]
 [ 114  124   28 ...    0    0    0]
 [   5   91    6 ...    0    0    0]]


In [64]:
print(w2v_weights.shape)

(5414, 300)


In [65]:
print(embedding_size)
test = []

for x in range(300):
    test.append(0)
testNp = np.array(test);

w2v_weights = np.vstack([w2v_weights,testNp])

model = Sequential()
print(y_train)
# Keras Embedding layer with Word2Vec weights initialization
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    weights=[embed_matrix],
                    input_length=45,
                       embeddings_initializer=Constant(embed_matrix)))

model.add(Bidirectional(LSTM(100)))
model.add(Dense(num_classes, activation='softmax'))
model.add(Flatten())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=5, batch_size=64,
                    validation_data=(X_validation, y_validation), verbose=1)

300
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Epoch 1/5
107/107 [==============================] - 12s 94ms/step - loss: 0.6180 - accuracy: 0.6963 - val_loss: 0.6053 - val_accuracy: 0.7079
Epoch 2/5
107/107 [==============================] - 9s 86ms/step - loss: 0.5910 - accuracy: 0.7009 - val_loss: 0.6020 - val_accuracy: 0.7091
Epoch 3/5
107/107 [==============================] - 9s 86ms/step - loss: 0.4978 - accuracy: 0.7494 - val_loss: 0.6269 - val_accuracy: 0.6565
Epoch 4/5
107/107 [==============================] - 9s 86ms/step - loss: 0.4131 - accuracy: 0.7980 - val_loss: 0.7128 - val_accuracy: 0.6600
Epoch 5/5
107/107 [==============================] - 9s 87ms/step - loss: 0.3574 - accuracy: 0.8262 - val_loss: 0.7957 - val_accuracy: 0.6332
